# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))
    all_train_edges = adj_train.nonzero()
    out_train_edges = [[all_train_edges[0][i],all_train_edges[1][i]] for i in range(all_train_edges[0].shape[0])]
    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return np.array(out_train_edges), val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    train_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0], replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [6]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(task=2)

# 定义模型

In [7]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [8]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 5.0
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 10
N = 29.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Cornell'

In [9]:
all_best_auc = []

all_best_ap = []

for i in range(10):
    
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(200):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 149.6782
Epoch: 01, Loss: 149.5437
Epoch: 02, Loss: 149.2892
Epoch: 03, Loss: 148.9286
Epoch: 04, Loss: 148.4751
Epoch: 05, Loss: 147.9409
Epoch: 06, Loss: 147.3372
Epoch: 07, Loss: 146.6742
Epoch: 08, Loss: 145.9612
Epoch: 09, Loss: 145.2063
AP SCORE：  0.5201054088852382
AUC SCORE:  0.5185185185185185
Epoch: 00, Loss: 144.4168
Epoch: 01, Loss: 144.3947
Epoch: 02, Loss: 144.3528
Epoch: 03, Loss: 144.2934
Epoch: 04, Loss: 144.2185
Epoch: 05, Loss: 144.1301
Epoch: 06, Loss: 144.0298
Epoch: 07, Loss: 143.9192
Epoch: 08, Loss: 143.7999
Epoch: 09, Loss: 143.6731
AP SCORE：  0.526146061678342
AUC SCORE:  0.5185185185185185
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 143.5400
Epoch: 01, Loss: 143.4244
Epoch: 02, Loss: 143.2052
Epoch: 03, Loss: 142.8936
Epoch: 04, Loss: 142.4999
Epoch: 05, Loss: 142.0335
Epoch: 06, Loss: 141.5032
Epoch: 07, Loss: 140.9171
Epoch: 08, Loss: 140.2826
E

Epoch: 07, Loss: 88.2049
Epoch: 08, Loss: 87.9178
Epoch: 09, Loss: 87.6128
AP SCORE：  0.7358516526670684
AUC SCORE:  0.6944444444444444
Epoch: 00, Loss: 87.2930
Epoch: 01, Loss: 87.2749
Epoch: 02, Loss: 87.2407
Epoch: 03, Loss: 87.1924
Epoch: 04, Loss: 87.1319
Epoch: 05, Loss: 87.0609
Epoch: 06, Loss: 86.9813
Epoch: 07, Loss: 86.8946
Epoch: 08, Loss: 86.8020
Epoch: 09, Loss: 86.7051
AP SCORE：  0.7327991496145655
AUC SCORE:  0.691358024691358
######################### 模块循环 ： 14 ##########################
Epoch: 00, Loss: 86.6049
Epoch: 01, Loss: 86.5571
Epoch: 02, Loss: 86.4665
Epoch: 03, Loss: 86.3378
Epoch: 04, Loss: 86.1754
Epoch: 05, Loss: 85.9832
Epoch: 06, Loss: 85.7651
Epoch: 07, Loss: 85.5243
Epoch: 08, Loss: 85.2639
Epoch: 09, Loss: 84.9869
AP SCORE：  0.7347493598981091
AUC SCORE:  0.691358024691358
Epoch: 00, Loss: 84.6957
Epoch: 01, Loss: 84.6809
Epoch: 02, Loss: 84.6529
Epoch: 03, Loss: 84.6134
Epoch: 04, Loss: 84.5639
Epoch: 05, Loss: 84.5057
Epoch: 06, Loss: 84.4403
Epoch:

AP SCORE：  0.7645193137096025
AUC SCORE:  0.7253086419753086
Epoch: 00, Loss: 67.3965
Epoch: 01, Loss: 67.3925
Epoch: 02, Loss: 67.3848
Epoch: 03, Loss: 67.3740
Epoch: 04, Loss: 67.3604
Epoch: 05, Loss: 67.3443
Epoch: 06, Loss: 67.3261
Epoch: 07, Loss: 67.3060
Epoch: 08, Loss: 67.2843
Epoch: 09, Loss: 67.2613
AP SCORE：  0.7645193137096025
AUC SCORE:  0.7253086419753086
######################### 模块循环 ： 26 ##########################
Epoch: 00, Loss: 67.2372
Epoch: 01, Loss: 67.2176
Epoch: 02, Loss: 67.1805
Epoch: 03, Loss: 67.1277
Epoch: 04, Loss: 67.0609
Epoch: 05, Loss: 66.9819
Epoch: 06, Loss: 66.8920
Epoch: 07, Loss: 66.7926
Epoch: 08, Loss: 66.6850
Epoch: 09, Loss: 66.5702
AP SCORE：  0.7660564239423648
AUC SCORE:  0.728395061728395
Epoch: 00, Loss: 66.4493
Epoch: 01, Loss: 66.4455
Epoch: 02, Loss: 66.4383
Epoch: 03, Loss: 66.4280
Epoch: 04, Loss: 66.4151
Epoch: 05, Loss: 66.3999
Epoch: 06, Loss: 66.3826
Epoch: 07, Loss: 66.3635
Epoch: 08, Loss: 66.3430
Epoch: 09, Loss: 66.3212
AP SC

Epoch: 04, Loss: 58.5792
Epoch: 05, Loss: 58.5694
Epoch: 06, Loss: 58.5583
Epoch: 07, Loss: 58.5461
Epoch: 08, Loss: 58.5330
Epoch: 09, Loss: 58.5190
AP SCORE：  0.7419966345264726
AUC SCORE:  0.7345679012345679
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 58.5043
Epoch: 01, Loss: 58.4932
Epoch: 02, Loss: 58.4722
Epoch: 03, Loss: 58.4423
Epoch: 04, Loss: 58.4044
Epoch: 05, Loss: 58.3595
Epoch: 06, Loss: 58.3083
Epoch: 07, Loss: 58.2517
Epoch: 08, Loss: 58.1901
Epoch: 09, Loss: 58.1243
AP SCORE：  0.7419966345264726
AUC SCORE:  0.7345679012345679
Epoch: 00, Loss: 58.0548
Epoch: 01, Loss: 58.0523
Epoch: 02, Loss: 58.0477
Epoch: 03, Loss: 58.0411
Epoch: 04, Loss: 58.0328
Epoch: 05, Loss: 58.0230
Epoch: 06, Loss: 58.0119
Epoch: 07, Loss: 57.9997
Epoch: 08, Loss: 57.9864
Epoch: 09, Loss: 57.9723
AP SCORE：  0.7433855234153615
AUC SCORE:  0.7376543209876543
######################### 模块循环 ： 39 ##########################
Epoch: 00, Loss: 57.9575
Epoch: 01, Loss:

Epoch: 09, Loss: 52.0543
AP SCORE：  0.7557537196153261
AUC SCORE:  0.7530864197530864
Epoch: 00, Loss: 52.0055
Epoch: 01, Loss: 52.0037
Epoch: 02, Loss: 52.0004
Epoch: 03, Loss: 51.9957
Epoch: 04, Loss: 51.9898
Epoch: 05, Loss: 51.9829
Epoch: 06, Loss: 51.9751
Epoch: 07, Loss: 51.9666
Epoch: 08, Loss: 51.9575
Epoch: 09, Loss: 51.9479
AP SCORE：  0.7557537196153261
AUC SCORE:  0.7530864197530864
######################### 模块循环 ： 52 ##########################
Epoch: 00, Loss: 51.9380
Epoch: 01, Loss: 51.9303
Epoch: 02, Loss: 51.9159
Epoch: 03, Loss: 51.8953
Epoch: 04, Loss: 51.8693
Epoch: 05, Loss: 51.8384
Epoch: 06, Loss: 51.8033
Epoch: 07, Loss: 51.7644
Epoch: 08, Loss: 51.7222
Epoch: 09, Loss: 51.6772
AP SCORE：  0.7512082650698715
AUC SCORE:  0.75
Epoch: 00, Loss: 51.6296
Epoch: 01, Loss: 51.6279
Epoch: 02, Loss: 51.6246
Epoch: 03, Loss: 51.6200
Epoch: 04, Loss: 51.6143
Epoch: 05, Loss: 51.6075
Epoch: 06, Loss: 51.5998
Epoch: 07, Loss: 51.5915
Epoch: 08, Loss: 51.5825
Epoch: 09, Loss: 5

...

Epoch: 05, Loss: 28.0859
Epoch: 06, Loss: 28.0759
Epoch: 07, Loss: 28.0648
Epoch: 08, Loss: 28.0527
Epoch: 09, Loss: 28.0398
AP SCORE：  0.6489931805340178
AUC SCORE:  0.5771604938271605
Epoch: 00, Loss: 28.0261
Epoch: 01, Loss: 28.0258
Epoch: 02, Loss: 28.0253
Epoch: 03, Loss: 28.0245
Epoch: 04, Loss: 28.0236
Epoch: 05, Loss: 28.0226
Epoch: 06, Loss: 28.0214
Epoch: 07, Loss: 28.0202
Epoch: 08, Loss: 28.0189
Epoch: 09, Loss: 28.0176
AP SCORE：  0.6489931805340178
AUC SCORE:  0.5771604938271605
######################### 模块循环 ： 145 ##########################
Epoch: 00, Loss: 28.0162
Epoch: 01, Loss: 28.0141
Epoch: 02, Loss: 28.0099
Epoch: 03, Loss: 28.0041
Epoch: 04, Loss: 27.9966
Epoch: 05, Loss: 27.9878
Epoch: 06, Loss: 27.9777
Epoch: 07, Loss: 27.9665
Epoch: 08, Loss: 27.9544
Epoch: 09, Loss: 27.9413
AP SCORE：  0.6489931805340178
AUC SCORE:  0.5771604938271605
Epoch: 00, Loss: 27.9275
Epoch: 01, Loss: 27.9272
Epoch: 02, Loss: 27.9267
Epoch: 03, Loss: 27.9260
Epoch: 04, Loss: 27.9252
Epo

Epoch: 01, Loss: 26.7664
Epoch: 02, Loss: 26.7659
Epoch: 03, Loss: 26.7653
Epoch: 04, Loss: 26.7646
Epoch: 05, Loss: 26.7637
Epoch: 06, Loss: 26.7628
Epoch: 07, Loss: 26.7618
Epoch: 08, Loss: 26.7608
Epoch: 09, Loss: 26.7598
AP SCORE：  0.6532274832860409
AUC SCORE:  0.5802469135802468
######################### 模块循环 ： 157 ##########################
Epoch: 00, Loss: 26.7588
Epoch: 01, Loss: 26.7565
Epoch: 02, Loss: 26.7522
Epoch: 03, Loss: 26.7460
Epoch: 04, Loss: 26.7382
Epoch: 05, Loss: 26.7291
Epoch: 06, Loss: 26.7186
Epoch: 07, Loss: 26.7071
Epoch: 08, Loss: 26.6946
Epoch: 09, Loss: 26.6813
AP SCORE：  0.6532274832860409
AUC SCORE:  0.5802469135802468
Epoch: 00, Loss: 26.6673
Epoch: 01, Loss: 26.6670
Epoch: 02, Loss: 26.6666
Epoch: 03, Loss: 26.6660
Epoch: 04, Loss: 26.6652
Epoch: 05, Loss: 26.6644
Epoch: 06, Loss: 26.6634
Epoch: 07, Loss: 26.6625
Epoch: 08, Loss: 26.6615
Epoch: 09, Loss: 26.6605
AP SCORE：  0.6532274832860409
AUC SCORE:  0.5802469135802468
######################### 模块

Epoch: 02, Loss: 25.7065
Epoch: 03, Loss: 25.7019
Epoch: 04, Loss: 25.6960
Epoch: 05, Loss: 25.6890
Epoch: 06, Loss: 25.6811
Epoch: 07, Loss: 25.6723
Epoch: 08, Loss: 25.6628
Epoch: 09, Loss: 25.6527
AP SCORE：  0.6613954903405993
AUC SCORE:  0.5895061728395061
Epoch: 00, Loss: 25.6421
Epoch: 01, Loss: 25.6418
Epoch: 02, Loss: 25.6414
Epoch: 03, Loss: 25.6408
Epoch: 04, Loss: 25.6401
Epoch: 05, Loss: 25.6393
Epoch: 06, Loss: 25.6384
Epoch: 07, Loss: 25.6375
Epoch: 08, Loss: 25.6365
Epoch: 09, Loss: 25.6355
AP SCORE：  0.6613954903405993
AUC SCORE:  0.5895061728395061
######################### 模块循环 ： 170 ##########################
Epoch: 00, Loss: 25.6346
Epoch: 01, Loss: 25.6328
Epoch: 02, Loss: 25.6296
Epoch: 03, Loss: 25.6250
Epoch: 04, Loss: 25.6192
Epoch: 05, Loss: 25.6123
Epoch: 06, Loss: 25.6045
Epoch: 07, Loss: 25.5958
Epoch: 08, Loss: 25.5865
Epoch: 09, Loss: 25.5765
AP SCORE：  0.6613954903405993
AUC SCORE:  0.5895061728395061
Epoch: 00, Loss: 25.5660
Epoch: 01, Loss: 25.5657
Epo

Epoch: 08, Loss: 24.7334
Epoch: 09, Loss: 24.7246
AP SCORE：  0.6802163961771659
AUC SCORE:  0.6141975308641976
Epoch: 00, Loss: 24.7154
Epoch: 01, Loss: 24.7152
Epoch: 02, Loss: 24.7147
Epoch: 03, Loss: 24.7140
Epoch: 04, Loss: 24.7132
Epoch: 05, Loss: 24.7122
Epoch: 06, Loss: 24.7112
Epoch: 07, Loss: 24.7101
Epoch: 08, Loss: 24.7089
Epoch: 09, Loss: 24.7078
AP SCORE：  0.6813186889461254
AUC SCORE:  0.617283950617284
######################### 模块循环 ： 182 ##########################
Epoch: 00, Loss: 24.7066
Epoch: 01, Loss: 24.7052
Epoch: 02, Loss: 24.7024
Epoch: 03, Loss: 24.6985
Epoch: 04, Loss: 24.6935
Epoch: 05, Loss: 24.6877
Epoch: 06, Loss: 24.6811
Epoch: 07, Loss: 24.6738
Epoch: 08, Loss: 24.6660
Epoch: 09, Loss: 24.6576
AP SCORE：  0.6788258114532478
AUC SCORE:  0.6141975308641975
Epoch: 00, Loss: 24.6489
Epoch: 01, Loss: 24.6487
Epoch: 02, Loss: 24.6482
Epoch: 03, Loss: 24.6476
Epoch: 04, Loss: 24.6468
Epoch: 05, Loss: 24.6460
Epoch: 06, Loss: 24.6450
Epoch: 07, Loss: 24.6440
Epoc

Epoch: 06, Loss: 24.0712
Epoch: 07, Loss: 24.0704
Epoch: 08, Loss: 24.0696
Epoch: 09, Loss: 24.0688
AP SCORE：  0.6802531280979558
AUC SCORE:  0.6172839506172839
######################### 模块循环 ： 194 ##########################
Epoch: 00, Loss: 24.0680
Epoch: 01, Loss: 24.0670
Epoch: 02, Loss: 24.0651
Epoch: 03, Loss: 24.0625
Epoch: 04, Loss: 24.0591
Epoch: 05, Loss: 24.0551
Epoch: 06, Loss: 24.0507
Epoch: 07, Loss: 24.0457
Epoch: 08, Loss: 24.0403
Epoch: 09, Loss: 24.0346
AP SCORE：  0.6802531280979558
AUC SCORE:  0.6172839506172839
Epoch: 00, Loss: 24.0285
Epoch: 01, Loss: 24.0284
Epoch: 02, Loss: 24.0280
Epoch: 03, Loss: 24.0276
Epoch: 04, Loss: 24.0270
Epoch: 05, Loss: 24.0263
Epoch: 06, Loss: 24.0256
Epoch: 07, Loss: 24.0248
Epoch: 08, Loss: 24.0240
Epoch: 09, Loss: 24.0232
AP SCORE：  0.6802531280979558
AUC SCORE:  0.6172839506172839
######################### 模块循环 ： 195 ##########################
Epoch: 00, Loss: 24.0223
Epoch: 01, Loss: 24.0214
Epoch: 02, Loss: 24.0195
Epoch: 03, Los

In [10]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())

"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.7916840729535329
STD AP :  0.07898054747030425
MEAN AUC :  0.7796296296296296
STD AUC :  0.08139258150679826


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'